In [30]:
# todo 学習結果を使って運用するための環境を作る。

# 選手情報・過去レース情報から3連単舟券120種をクラス分類する
# LGBMでやってみる。
# todo:bayseの方と、インプットの与え方をあわせる。今はOne-Hotが一致していない。
# 的中予想確率x%以上のうち、オッズで見合うものだけ買った場合のシミュレーションをする。

# このnoteでは情報を増やしてみる。過去100レースの着順分布をいれる。m1rank1,m1rank2,…m1rank6,m2rank1,…,m6rank6
# 期間を変えるときには、num_leavesも調整する。

# 汎用ライブラリのimport
import sys
import os
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import statsmodels.api as sm
import math
import tensorflow as tf
import collections
from datetime import datetime

import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import csv
from itertools import chain

In [34]:
# 自作ライブラリのimport
if os.environ['BR_HOME']+"/boatrace" not in sys.path:
    sys.path.append(os.environ['BR_HOME']+"/boatrace")
#print(sys.path)

from setup.myUtil import dbHandler

# model保存パスのセット
model_path=os.path.join(os.environ['BR_HOME'],"boatrace/models")


In [3]:
# 舟券の配列を取得
funakenList=[]
with open(os.environ['BR_HOME']+'/boatrace/config/3t_list.dat') as f:
    reader=csv.reader(f)
    for row in reader:
        funakenList.append(row)
funakenID = [i for i in range(120)]
funakenDict=dict(zip(funakenList[0],funakenID))
#print(funakenDict)

In [4]:
# 分析期間の指定は一旦ここでまとめてみる。
trainStartDate="20170901"
trainEndDate="20181231"
# test はtrainからsplitする
testStartDate="20190101"
testEndDate="20190531"

In [5]:
dbh=dbHandler.getDBHandle()
#dbHandler.closeDBHandle(dbh)

In [6]:
# trainの元データを取得
with dbh.cursor() as cursor:
    sel_sql = "select * from raceabst_forml_rentai_v \
               where raceDate between '%s' and '%s' \
               order by raceId "\
               % (trainStartDate,trainEndDate)
    cursor.execute(sel_sql)
    loadList=cursor.fetchall()
print("traindata:",len(loadList))


traindata: 71699


In [7]:
df = pd.io.json.json_normalize(loadList)
df.head()

,funaken,l1Fcnt,l1boat2r,l1boat3r,l1motor2r,l1motor3r,l1oldavgstdev,l1oldavgsttime,l1oldrank1,l1oldrank2,...,l6oldrank4,l6oldrank5,l6oldrank6,l6rank,l6score,odds,raceDate,raceId,raceWaveHeight,raceWindSpeed
0,1-2-5,0,0.2632,0.3684,0.3158,0.4737,-0.02765,0.12900,0.26,0.25,...,0.10,0.06,0.09,A1,2.973712,22.8,2017-09-01,20170901-05-01,3,6.0
1,2-3-6,0,0.4737,0.7368,0.2963,0.4074,-0.06915,0.16990,0.41,0.15,...,0.14,0.13,0.08,A1,4.878168,92.7,2017-09-01,20170901-05-02,3,4.0
2,3-1-6,0,0.3500,0.6500,0.5556,0.5556,-0.02338,0.12890,0.42,0.27,...,0.11,0.11,0.02,A1,0.877197,54.5,2017-09-01,20170901-05-03,2,4.0
3,1-5-2,0,0.4444,0.5556,0.2667,0.4000,-0.01867,0.14760,0.38,0.27,...,0.16,0.15,0.09,A1,1.284942,13.1,2017-09-01,20170901-05-04,3,5.0
4,1-2-4,0,0.3000,0.5500,0.3077,0.5000,0.00234,0.15212,0.25,0.28,...,0.17,0.10,0.05,A1,2.240696,12.0,2017-09-01,20170901-05-05,2,3.0


In [8]:
# 入力のデータ整形
#xdf=df.drop(['funaken','odds','raceId','raceDate'],axis=1)
#xdf=pd.get_dummies(xdf,columns=['l1rank','l2rank','l3rank','l4rank','l5rank','l6rank'])
#xdf.head()

# 入力のデータ整形
xdf=df.drop(['funaken','odds','raceId','raceDate'],axis=1)
# オッズから作ったスコアは効きすぎるので捨ててみる
xdf=xdf.drop(['l1score','l2score','l3score','l4score','l5score','l6score'],axis=1)
#xdf=xdf.drop(['l1Fcnt','l2Fcnt','l3Fcnt','l4Fcnt','l5Fcnt','l6Fcnt'],axis=1)
#xdf=xdf.drop(['l1oldavgstdev','l2oldavgstdev','l3oldavgstdev','l4oldavgstdev','l5oldavgstdev','l6oldavgstdev'],axis=1)

#xdf=pd.get_dummies(xdf,columns=['l1rank','l2rank','l3rank','l4rank','l5rank','l6rank'])
rankLabel=LabelEncoder()
rankLabel=rankLabel.fit(xdf['l1rank'])
xdf['l1rank']=rankLabel.transform(xdf['l1rank'])
xdf['l2rank']=rankLabel.transform(xdf['l2rank'])
xdf['l3rank']=rankLabel.transform(xdf['l3rank'])
xdf['l4rank']=rankLabel.transform(xdf['l4rank'])
xdf['l5rank']=rankLabel.transform(xdf['l5rank'])
xdf['l6rank']=rankLabel.transform(xdf['l6rank'])
xdf.head()


,l1Fcnt,l1boat2r,l1boat3r,l1motor2r,l1motor3r,l1oldavgstdev,l1oldavgsttime,l1oldrank1,l1oldrank2,l1oldrank3,...,l6oldavgsttime,l6oldrank1,l6oldrank2,l6oldrank3,l6oldrank4,l6oldrank5,l6oldrank6,l6rank,raceWaveHeight,raceWindSpeed
0,0,0.2632,0.3684,0.3158,0.4737,-0.02765,0.12900,0.26,0.25,0.15,...,0.1587,0.28,0.21,0.26,0.10,0.06,0.09,0,3,6.0
1,0,0.4737,0.7368,0.2963,0.4074,-0.06915,0.16990,0.41,0.15,0.21,...,0.1307,0.23,0.22,0.20,0.14,0.13,0.08,0,3,4.0
2,0,0.3500,0.6500,0.5556,0.5556,-0.02338,0.12890,0.42,0.27,0.11,...,0.1754,0.33,0.23,0.20,0.11,0.11,0.02,0,2,4.0
3,0,0.4444,0.5556,0.2667,0.4000,-0.01867,0.14760,0.38,0.27,0.12,...,0.1642,0.23,0.22,0.15,0.16,0.15,0.09,0,3,5.0
4,0,0.3000,0.5500,0.3077,0.5000,0.00234,0.15212,0.25,0.28,0.18,...,0.1508,0.34,0.19,0.15,0.17,0.10,0.05,0,2,3.0


In [9]:
xdf.describe()



,l1Fcnt,l1boat2r,l1boat3r,l1motor2r,l1motor3r,l1oldavgstdev,l1oldavgsttime,l1oldrank1,l1oldrank2,l1oldrank3,...,l6oldavgsttime,l6oldrank1,l6oldrank2,l6oldrank3,l6oldrank4,l6oldrank5,l6oldrank6,l6rank,raceWaveHeight,raceWindSpeed
count,71699.000000,71699.000000,71699.000000,71699.000000,71699.000000,71699.000000,71699.000000,71699.000000,71699.000000,71699.000000,...,71699.000000,71699.000000,71699.000000,71699.000000,71699.000000,71699.000000,71699.000000,71699.000000,71699.000000,71699.000000
mean,0.146515,0.322848,0.484029,0.325478,0.486800,-0.002357,0.180134,0.189151,0.178933,0.171875,...,0.184803,0.147644,0.149928,0.152595,0.156500,0.164145,0.186710,1.477287,2.535963,2.777012
std,0.364041,0.105688,0.133595,0.110905,0.139719,0.037452,0.044663,0.096203,0.056665,0.046874,...,0.049585,0.102032,0.074287,0.064682,0.060103,0.067228,0.142276,0.955244,2.065998,1.869531
min,0.000000,0.000000,0.000000,0.000000,0.000000,-0.142670,0.099100,0.000000,0.000000,0.000000,...,0.030000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.283000,0.444400,0.277800,0.438100,-0.022870,0.154000,0.110000,0.140000,0.140000,...,0.157800,0.070000,0.110000,0.120000,0.120000,0.120000,0.100000,1.000000,1.000000,1.000000
50%,0.000000,0.333300,0.500000,0.331900,0.500000,-0.008520,0.170100,0.190000,0.180000,0.170000,...,0.174500,0.140000,0.160000,0.160000,0.160000,0.160000,0.150000,2.000000,2.000000,3.000000
75%,0.000000,0.378400,0.552000,0.385000,0.563600,0.006500,0.189600,0.260000,0.220000,0.200000,...,0.195200,0.220000,0.200000,0.200000,0.200000,0.210000,0.220000,2.000000,3.000000,4.000000
max,2.000000,1.000000,1.000000,1.000000,1.000000,0.302680,0.541800,0.630000,0.400000,0.360000,...,3.450000,0.630000,0.390000,0.370000,0.400000,0.430000,0.920000,3.000000,30.000000,15.000000


In [10]:
# ファイルから作った辞書で変換する
ydf=df['funaken']
ydf=pd.DataFrame(ydf.replace(funakenDict))
ydf['funaken']=ydf['funaken'].astype(int)
#print(ydf.head())
print(ydf['funaken'].dtype)

int64


In [11]:
# 重み付けのため、オッズのリストを作る
odf=df['odds'].values
#odf=pd.DataFrame(df['odds'])
#odf.describe()
#print(odf)

In [12]:
X_train, X_test, y_train, y_test,o_train,o_test = train_test_split(xdf, ydf,odf)
print("X_train,X_test:",len(X_train),len(X_test))

X_train,X_test: 53774 17925


In [13]:
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

In [14]:
lgbm_params = {
    # 多値分類問題
    'objective': 'multiclass',
    'num_boost_round':250,
    # クラス数は 120
    'num_class': 120,
    #'class_weight':'balanced',
    #'random_state':999,
    # 以下、ハイパーパラメタ
    'max_depth':2,
    'num_leaves':3,
    'min_data_in_leaf':300,
    # 正則化
    'reg_alpha':6.161,
    'reg_lambda':8.977,
}


In [15]:
lgb.LGBMClassifier()

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.1, max_depth=-1,
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
        random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [16]:
model = lgb.train(lgbm_params, lgb_train, valid_sets=lgb_eval)

/home/ec2-user/.pyenv/versions/anaconda3-2018.12/lib/python3.6/site-packages/lightgbm/engine.py:113: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's multi_logloss: 4.68782
[2]	valid_0's multi_logloss: 4.61334
[3]	valid_0's multi_logloss: 4.55377
[4]	valid_0's multi_logloss: 4.50444
[5]	valid_0's multi_logloss: 4.46259
[6]	valid_0's multi_logloss: 4.4265
[7]	valid_0's multi_logloss: 4.39439
[8]	valid_0's multi_logloss: 4.36642
[9]	valid_0's multi_logloss: 4.34116
[10]	valid_0's multi_logloss: 4.3188
[11]	valid_0's multi_logloss: 4.29831
[12]	valid_0's multi_logloss: 4.27986
[13]	valid_0's multi_logloss: 4.26286
[14]	valid_0's multi_logloss: 4.24765
[15]	valid_0's multi_logloss: 4.23333
[16]	valid_0's multi_logloss: 4.22029
[17]	valid_0's multi_logloss: 4.20808
[18]	valid_0's multi_logloss: 4.1969
[19]	valid_0's multi_logloss: 4.1866
[20]	valid_0's multi_logloss: 4.17677
[21]	valid_0's multi_logloss: 4.16775
[22]	valid_0's multi_logloss: 4.15946
[23]	valid_0's multi_logloss: 4.15165
[24]	valid_0's multi_logloss: 4.14409
[25]	valid_0's multi_logloss: 4.13717
[26]	valid_0's multi_logloss: 4.13072
[27]	valid_0's multi_logl

[215]	valid_0's multi_logloss: 3.96222
[216]	valid_0's multi_logloss: 3.9622
[217]	valid_0's multi_logloss: 3.96208
[218]	valid_0's multi_logloss: 3.96198
[219]	valid_0's multi_logloss: 3.9619
[220]	valid_0's multi_logloss: 3.96182
[221]	valid_0's multi_logloss: 3.96184
[222]	valid_0's multi_logloss: 3.96172
[223]	valid_0's multi_logloss: 3.96167
[224]	valid_0's multi_logloss: 3.96156
[225]	valid_0's multi_logloss: 3.96146
[226]	valid_0's multi_logloss: 3.96142
[227]	valid_0's multi_logloss: 3.96131
[228]	valid_0's multi_logloss: 3.96124
[229]	valid_0's multi_logloss: 3.96118
[230]	valid_0's multi_logloss: 3.96109
[231]	valid_0's multi_logloss: 3.961
[232]	valid_0's multi_logloss: 3.9609
[233]	valid_0's multi_logloss: 3.96079
[234]	valid_0's multi_logloss: 3.96076
[235]	valid_0's multi_logloss: 3.96071
[236]	valid_0's multi_logloss: 3.96071
[237]	valid_0's multi_logloss: 3.96068
[238]	valid_0's multi_logloss: 3.9606
[239]	valid_0's multi_logloss: 3.96064
[240]	valid_0's multi_logloss: 

In [35]:
## save_model
model_name=os.path.join(datetime.now().strftime("%Y%m%d%H%M%S"))
print(model_name)
model.save_model(os.path.join(model_path,model_name+'.txt'))


20190714063621


In [17]:
y_pred = model.predict(X_test, num_iteration=model.best_iteration)
y_pred_max = np.argmax(y_pred, axis=1)  # 最尤と判断したクラスの値にする

# 精度 (Accuracy) を計算する
accuracy = sum(y_test['funaken'] == y_pred_max) / len(y_test)
print("accuracy:",accuracy)

# 回収率を計算
res=0
for i in range(len(y_test)):
    if (y_test.iloc[i])['funaken']==y_pred_max[i]:
        #print("i:",i,"result:",y_test[i],"forecast:",y_pred_max[i],"forecastProb:",y_pred[i][y_pred_max[i]],"return:",o_test[i],"expect:",y_pred[i][y_pred_max[i]]*o_test[i])
        res += o_test[i] -1
    else:
        #print("i:",i,"result:",y_test[i],"forecast:",y_pred_max[i],"forecastProb:",y_pred[i][y_pred_max[i]],"return:",o_test[i],"expect:",y_pred[i][y_pred_max[i]]*o_test[i])
        pass
print("resultReturn:",res/len(y_test))


accuracy: 0.08117154811715481
resultReturn: 0.7193751743375179


In [18]:
print(len(y_pred))
# オッズを見て判断する場合
resAmount=0
buyAmount=0

dfIndexList=y_test['funaken'].index
for i in range(len(y_test)):
    raceId = df.iloc[dfIndexList[i]]['raceId']
    #print(raceId)
    with dbh.cursor() as cursor:
        sel_sql = "select funaken,odds from raceodds \
                   where oddsType = '3t' \
                   and raceId = '%s' \
                   order by funaken" \
                   % (raceId)
        cursor.execute(sel_sql)
        loadList=pd.DataFrame(cursor.fetchall())
        loadList=pd.DataFrame(loadList.replace(funakenDict))
    #print((loadList[loadList['funaken']==0]['odds'] * y_pred[i][0] ).values[0] )
    #if ( (loadList[loadList['funaken']==0]['odds'] * y_pred[i][0] ).values[0] > 1) :
    #    print("ok")
    #print(y_pred[i][0])
    #print(raceId)
    for j in range(120):
        if y_pred[i][j]> 0.10 and ( (y_pred[i][j] * (loadList[loadList['funaken']==j]['odds'])).values[0] > 1.5) :
            print("buy:",raceId,i,j,loadList[loadList['funaken']==j]['odds'].values[0])
            buyAmount+=1
            if y_test['funaken'].iloc[i]==j:
            #print("i:",i,"result:",y_test[i],"forecast:",y_pred_max[i],"forecastProb:",y_pred[i][y_pred_max[i]],"return:",o_test[i],"expect:",y_pred[i][y_pred_max[i]]*o_test[i])
                print("☆hit!☆:",raceId,j,loadList[loadList['funaken']==j]['odds'].values[0])
                resAmount += o_test[i]
            else:
            #print("i:",i,"result:",y_test[i],"forecast:",y_pred_max[i],"forecastProb:",y_pred[i][y_pred_max[i]],"return:",o_test[i],"expect:",y_pred[i][y_pred_max[i]]*o_test[i])
                pass
print("resultReturn:",resAmount/buyAmount)


17925
buy: 20180407-10-03 57 0 12.5
buy: 20180328-14-10 86 0 19.3
buy: 20180505-04-09 200 1 15.7
buy: 20170926-19-05 259 0 16.0
buy: 20170918-20-08 299 4 16.1
buy: 20180529-19-07 308 0 22.6
buy: 20180826-17-12 310 0 14.1
buy: 20180511-16-01 315 4 16.4
buy: 20180504-02-07 330 0 17.3
buy: 20180305-17-12 403 0 21.2
buy: 20171202-23-03 438 0 16.8
buy: 20171116-10-05 439 23 17.7
buy: 20180924-20-10 452 1 13.1
buy: 20171118-04-05 466 0 17.9
buy: 20180822-23-03 474 0 18.8
buy: 20171109-04-05 476 0 15.8
buy: 20180606-10-03 509 6 10.7
☆hit!☆: 20180606-10-03 6 10.7
buy: 20180409-12-06 520 0 20.1
buy: 20180630-06-02 555 0 20.3
☆hit!☆: 20180630-06-02 0 20.3
buy: 20171215-23-02 581 0 23.5
buy: 20180316-21-01 718 0 15.0
buy: 20181225-07-05 730 0 29.0
buy: 20171012-11-04 795 0 20.8
buy: 20181210-04-08 837 0 15.3
buy: 20180618-21-01 877 1 19.7
buy: 20180918-03-08 903 0 16.1
buy: 20171223-06-10 918 0 18.6
buy: 20180130-13-07 932 8 18.4
buy: 20180529-03-08 943 0 16.0
buy: 20181026-11-01 965 4 13.5
buy: 

buy: 20180426-02-06 8687 0 14.0
buy: 20180318-16-05 8839 0 18.5
buy: 20171028-21-05 8907 0 15.3
buy: 20171206-19-03 8948 0 14.5
buy: 20170920-07-05 8999 0 28.8
buy: 20181111-15-12 9054 5 18.5
buy: 20181008-04-07 9078 0 11.1
☆hit!☆: 20181008-04-07 0 11.1
buy: 20180512-05-09 9111 0 16.5
buy: 20171103-02-05 9124 0 12.2
buy: 20180309-03-09 9168 4 11.8
buy: 20180104-22-09 9187 0 15.3
☆hit!☆: 20180104-22-09 0 15.3
buy: 20181128-04-12 9221 0 14.6
buy: 20180911-19-08 9291 0 17.5
buy: 20171202-11-01 9297 0 15.1
buy: 20180406-11-11 9334 0 13.7
buy: 20180708-15-08 9339 0 14.7
buy: 20180216-05-11 9399 4 19.9
buy: 20180608-08-08 9453 41 15.7
buy: 20180808-21-03 9463 0 13.7
buy: 20180603-06-02 9498 0 13.9
buy: 20180713-17-12 9547 0 17.5
buy: 20181204-05-04 9647 41 13.3
buy: 20180813-10-03 9696 4 15.2
buy: 20180807-05-06 9734 0 23.2
buy: 20180818-13-05 9757 2 28.6
buy: 20180818-04-08 9816 4 22.9
buy: 20170922-09-11 9857 0 12.9
☆hit!☆: 20170922-09-11 0 12.9
buy: 20180912-23-03 9865 0 17.8
buy: 2018060

buy: 20171010-03-06 17482 0 14.2
buy: 20171010-03-06 17482 20 14.6
buy: 20180621-01-07 17497 20 11.5
☆hit!☆: 20180621-01-07 20 11.5
buy: 20180225-04-09 17526 0 16.0
buy: 20180113-06-06 17529 20 11.0
buy: 20180509-06-07 17580 15 17.2
buy: 20180424-10-02 17616 1 13.5
buy: 20181005-16-06 17641 4 14.5
buy: 20181003-12-07 17675 6 12.3
buy: 20180227-07-09 17699 9 35.4
☆hit!☆: 20180227-07-09 9 35.4
buy: 20180223-01-08 17722 2 24.5
buy: 20180311-01-07 17725 1 21.6
buy: 20181221-19-01 17803 0 10.1
☆hit!☆: 20181221-19-01 0 10.1
buy: 20180326-08-09 17825 8 14.0
buy: 20181104-12-09 17847 23 11.7
resultReturn: 0.957258064516129


In [19]:
print("totalRace,buy,return,",len(y_test),buyAmount,resAmount ,resAmount/buyAmount)
# 現状でも100%はロバストに超えそう。←random_state入れてただけ。。random_state=999
# だがrrが1.2でもbuyが6/300レースだと、1000円投票しても一日だと約3000*1.2⇒利益600円なので、もっと回収率を上げるか母数増やす。
# totalRace,buy,return 28274 608 614.3
# totalRace,buy,return 28274 614 804.8

#項目を減らしてみる。
#totalRace,buy,return, 17925 151 153.29999999999998 1.0152317880794701

totalRace,buy,return, 17925 496 474.8 0.957258064516129


In [20]:
print(len(y_pred))
# オッズを見て判断する場合
resAmount=0
buyAmount=0

dfIndexList=y_test['funaken'].index
for i in range(len(y_test)):
    raceId = df.iloc[dfIndexList[i]]['raceId']
    #print(raceId)
    with dbh.cursor() as cursor:
        sel_sql = "select funaken,odds from raceodds \
                   where oddsType = '3t' \
                   and raceId = '%s' \
                   order by funaken" \
                   % (raceId)
        cursor.execute(sel_sql)
        loadList=pd.DataFrame(cursor.fetchall())
        loadList=pd.DataFrame(loadList.replace(funakenDict))
    #print((loadList[loadList['funaken']==0]['odds'] * y_pred[i][0] ).values[0] )
    #if ( (loadList[loadList['funaken']==0]['odds'] * y_pred[i][0] ).values[0] > 1) :
    #    print("ok")
    #print(y_pred[i][0])
    #print(raceId)
    for j in range(120):
        # y_predの閾値を下げてみる。
        if y_pred[i][j]> 0.05 and ( (y_pred[i][j] * (loadList[loadList['funaken']==j]['odds'])).values[0] > 1.5) :
            print("buy:",raceId,i,j,loadList[loadList['funaken']==j]['odds'].values[0],round(y_pred[i][j],3) )
            buyAmount+=1
            if y_test['funaken'].iloc[i]==j:
            #print("i:",i,"result:",y_test[i],"forecast:",y_pred_max[i],"forecastProb:",y_pred[i][y_pred_max[i]],"return:",o_test[i],"expect:",y_pred[i][y_pred_max[i]]*o_test[i])
                print("☆hit!☆:",raceId,j,loadList[loadList['funaken']==j]['odds'].values[0])
                resAmount += o_test[i]
            else:
            #print("i:",i,"result:",y_test[i],"forecast:",y_pred_max[i],"forecastProb:",y_pred[i][y_pred_max[i]],"return:",o_test[i],"expect:",y_pred[i][y_pred_max[i]]*o_test[i])
                pass
#print("resultReturn:",resAmount/buyAmount)
#print("totalRace,buy,return",len(y_test),buyAmount,resAmount )

# y_pred[i][j]> 0.05に閾値下げてみた。投票数はかなり増えているが、回収率が低く、トータルリターンは下がっている。
# ToDo：運用に向けてはこのバランスを調整してみるのはあり。
#resultReturn: 1.005279187817259
#totalRace,buy,return 28274 4925 4951.0

#resultReturn: 0.8838785046728972
#totalRace,buy,return 17925 6848 6052.8
#resultReturn: 0.8059009786989055
#totalRace,buy,return 17925 6948 5599.399999999996

#resultReturn: 0.8867549668874173
#totalRace,buy,return 17925 5587 4954.3

# 以下、入力項目を落としてみる。ボート、モーター、過去順位、スタートタイムだけ残す
#resultReturn: 0.9442170972776139
#totalRace,buy,return 17925 5767 5445.299999999999

#resultReturn: 0.8314920965780791
#totalRace,buy,return 17925 5757 4786.9000000000015

# パラメタ変更
#resultReturn: 0.86511287964918
#totalRace,buy,return 17925 6157 5326.500000000002

# 切り捨てに変更
#resultReturn: 0.8563023801542073
#totalRace,buy,return 17925 5966 5108.700000000001

17925
buy: 20180211-12-11 2 5 33.1 0.071
buy: 20171219-06-09 3 0 20.1 0.085
buy: 20180203-02-09 4 4 53.0 0.051
buy: 20180909-21-09 7 4 27.5 0.061
buy: 20180909-21-09 7 5 36.5 0.052
buy: 20180909-21-09 7 6 62.7 0.054
buy: 20171003-24-03 12 0 37.0 0.065
buy: 20171003-24-03 12 1 64.2 0.058
buy: 20171003-24-03 12 4 35.7 0.056
buy: 20171003-24-03 12 9 61.8 0.055
☆hit!☆: 20171003-24-03 9 61.8
buy: 20180722-19-06 17 47 76.2 0.06
buy: 20180414-06-06 19 20 43.5 0.058
buy: 20180909-06-05 21 4 37.1 0.052
buy: 20180618-14-09 23 0 37.9 0.051
buy: 20180805-01-05 27 0 21.3 0.097
buy: 20171001-02-04 31 0 32.1 0.066
buy: 20181222-15-12 32 11 33.8 0.058
buy: 20180620-22-03 38 8 39.8 0.057
buy: 20180720-09-02 41 0 20.7 0.074
buy: 20180720-09-02 41 4 36.4 0.05
buy: 20171027-19-07 44 0 24.9 0.064
buy: 20181024-13-08 48 16 38.8 0.068
buy: 20171209-03-11 49 1 27.5 0.071
buy: 20171209-03-11 49 11 25.2 0.066
buy: 20180407-10-03 57 0 12.5 0.128
buy: 20180407-10-03 57 1 20.8 0.073
☆hit!☆: 20180407-10-03 1 20.8
b

buy: 20181228-03-04 517 5 37.1 0.066
buy: 20181013-10-03 518 0 60.0 0.067
buy: 20181013-10-03 518 4 33.0 0.09
buy: 20181013-10-03 518 5 42.0 0.073
buy: 20171016-01-04 519 0 18.2 0.095
buy: 20171016-01-04 519 4 35.9 0.06
buy: 20171016-01-04 519 5 56.1 0.052
buy: 20180409-12-06 520 0 20.1 0.1
buy: 20180409-12-06 520 1 32.0 0.056
buy: 20171029-04-02 521 4 21.2 0.077
buy: 20171025-12-08 525 3 48.7 0.054
buy: 20171007-22-12 528 2 31.7 0.059
buy: 20180625-06-02 535 0 33.7 0.061
buy: 20180625-06-02 535 1 22.1 0.072
buy: 20171223-06-05 542 49 31.6 0.064
buy: 20180116-11-06 543 6 37.6 0.06
buy: 20180827-05-02 545 0 64.5 0.05
buy: 20180630-06-02 555 0 20.3 0.107
☆hit!☆: 20180630-06-02 0 20.3
buy: 20180630-06-02 555 4 40.8 0.054
buy: 20180114-04-02 557 21 63.3 0.063
buy: 20171101-05-09 561 0 33.4 0.095
buy: 20171101-05-09 561 2 45.9 0.067
buy: 20171101-05-09 561 4 26.6 0.073
buy: 20171226-04-06 566 2 29.7 0.068
buy: 20171115-06-02 568 21 26.0 0.064
☆hit!☆: 20171115-06-02 21 26.0
buy: 20180616-06-

buy: 20181027-16-08 1087 20 44.4 0.06
buy: 20181122-21-03 1088 0 30.5 0.055
buy: 20181122-21-03 1088 4 38.6 0.061
buy: 20180613-10-03 1091 3 30.9 0.067
buy: 20180106-11-11 1094 1 36.4 0.054
buy: 20170926-24-02 1096 0 14.3 0.109
buy: 20170926-24-02 1096 1 52.6 0.052
buy: 20170926-24-02 1096 2 31.8 0.064
buy: 20180319-09-07 1100 1 16.9 0.117
buy: 20180319-09-07 1100 21 26.6 0.08
buy: 20180311-23-05 1103 21 65.4 0.051
buy: 20171231-15-08 1104 5 21.4 0.081
buy: 20180228-22-06 1109 2 29.2 0.058
buy: 20180228-22-06 1109 4 14.9 0.121
buy: 20170904-01-04 1111 0 46.6 0.055
buy: 20170904-01-04 1111 1 46.3 0.069
buy: 20170904-01-04 1111 3 34.5 0.067
buy: 20171101-17-09 1121 2 20.6 0.087
buy: 20180726-02-03 1129 1 58.2 0.052
buy: 20180726-02-03 1129 4 30.2 0.061
buy: 20180523-02-06 1135 0 46.7 0.058
buy: 20180627-23-12 1136 9 29.2 0.054
buy: 20180627-23-12 1136 10 38.2 0.05
buy: 20180825-18-01 1137 3 36.6 0.052
buy: 20180809-07-05 1138 20 52.3 0.072
buy: 20180120-24-09 1145 2 28.0 0.068
buy: 20180

buy: 20180206-24-03 1589 4 26.3 0.073
buy: 20180814-22-05 1593 6 26.7 0.056
buy: 20180814-22-05 1593 12 40.9 0.052
buy: 20180423-05-06 1594 4 32.4 0.055
buy: 20180303-21-08 1595 0 15.5 0.134
buy: 20180102-03-07 1596 5 26.2 0.061
buy: 20180411-12-05 1600 8 41.2 0.061
buy: 20180819-11-08 1602 41 28.4 0.067
buy: 20171124-19-12 1606 0 22.3 0.1
buy: 20171124-19-12 1606 4 29.5 0.053
buy: 20171124-19-12 1606 8 26.1 0.062
buy: 20180216-17-05 1609 11 23.1 0.082
buy: 20180426-24-06 1610 47 31.6 0.082
buy: 20180724-04-01 1612 0 26.3 0.06
buy: 20180623-05-04 1614 36 19.5 0.121
buy: 20171130-13-06 1617 2 28.9 0.058
buy: 20171130-13-06 1617 4 28.8 0.058
buy: 20171223-05-05 1618 20 23.0 0.084
buy: 20171220-21-02 1625 20 41.5 0.054
buy: 20180714-06-11 1632 3 30.7 0.054
buy: 20180714-06-11 1632 8 27.4 0.061
buy: 20180826-13-01 1636 20 34.1 0.085
buy: 20180826-13-01 1636 31 29.7 0.052
buy: 20180525-13-02 1640 0 20.7 0.083
buy: 20180525-13-02 1640 4 32.1 0.076
buy: 20181117-19-08 1641 2 31.9 0.056
buy: 2

buy: 20171111-04-07 2122 4 45.6 0.063
buy: 20180920-19-03 2127 20 58.3 0.056
buy: 20181011-24-05 2128 0 18.8 0.098
buy: 20181011-24-05 2128 20 53.1 0.052
buy: 20180705-21-05 2133 6 107.8 0.051
buy: 20180208-05-11 2134 0 14.8 0.123
buy: 20180607-20-04 2135 8 29.1 0.059
buy: 20180521-07-09 2140 0 30.5 0.083
buy: 20180521-07-09 2140 1 39.4 0.062
buy: 20180320-02-02 2143 0 16.3 0.12
☆hit!☆: 20180320-02-02 0 16.3
buy: 20180906-21-10 2145 0 34.6 0.062
buy: 20180828-18-11 2147 0 22.8 0.073
buy: 20180828-18-11 2147 1 26.2 0.061
buy: 20180828-18-11 2147 8 27.1 0.06
buy: 20180806-05-03 2152 2 32.8 0.056
buy: 20180806-05-03 2152 12 45.1 0.056
buy: 20180805-23-11 2153 9 31.0 0.076
buy: 20180805-23-11 2153 11 95.1 0.051
buy: 20180927-17-06 2155 0 23.3 0.067
buy: 20171209-08-05 2156 9 28.8 0.054
buy: 20180130-13-08 2162 0 35.6 0.055
buy: 20180706-14-05 2177 1 17.0 0.104
buy: 20171020-20-07 2178 0 18.6 0.082
buy: 20180517-04-09 2186 5 40.6 0.052
buy: 20180517-04-09 2186 9 38.1 0.06
buy: 20180605-20-0

...

buy: 20180327-20-03 15323 1 30.0 0.061
buy: 20180130-01-07 15327 0 18.6 0.089
buy: 20180516-23-01 15332 0 20.4 0.083
buy: 20180516-23-01 15332 4 31.9 0.069
buy: 20180723-04-09 15336 0 21.3 0.088
buy: 20180723-04-09 15336 4 39.6 0.073
buy: 20180723-04-09 15336 5 36.2 0.058
buy: 20180822-22-12 15339 0 22.1 0.071
buy: 20180125-23-08 15343 0 20.0 0.076
buy: 20170902-05-06 15350 0 21.0 0.074
buy: 20170902-05-06 15350 2 33.8 0.055
buy: 20170902-05-06 15350 4 48.2 0.057
buy: 20171110-05-10 15352 0 30.0 0.06
☆hit!☆: 20171110-05-10 0 30.0
buy: 20181204-15-01 15353 16 20.5 0.104
buy: 20180713-21-05 15358 0 38.8 0.077
buy: 20180713-21-05 15358 20 34.8 0.05
buy: 20180205-03-11 15359 4 13.8 0.113
buy: 20180205-03-11 15359 5 35.4 0.058
buy: 20180407-02-10 15362 0 13.2 0.131
buy: 20170904-23-05 15376 0 22.9 0.072
buy: 20170904-23-05 15376 4 30.2 0.05
buy: 20180305-04-06 15381 1 20.0 0.079
buy: 20180616-20-06 15382 5 24.9 0.064
buy: 20180516-09-06 15383 0 26.9 0.058
buy: 20181109-06-07 15387 8 26.1 0.

buy: 20171015-21-03 15906 0 64.1 0.05
buy: 20171015-21-03 15906 3 132.5 0.056
buy: 20180129-02-02 15907 8 28.3 0.055
buy: 20180317-21-03 15908 12 44.7 0.051
buy: 20180726-02-10 15911 0 51.8 0.065
buy: 20180726-02-10 15911 3 79.9 0.051
buy: 20180408-22-02 15913 1 20.0 0.077
buy: 20180408-22-02 15913 21 46.4 0.052
buy: 20180115-01-07 15914 1 40.6 0.053
buy: 20181014-06-10 15920 0 21.0 0.085
buy: 20180505-23-04 15921 1 72.9 0.052
buy: 20180505-23-04 15921 5 59.1 0.053
buy: 20180505-23-04 15921 8 37.3 0.092
buy: 20180505-23-04 15921 9 27.3 0.063
buy: 20180114-02-03 15923 27 101.1 0.055
buy: 20170919-23-05 15924 1 42.6 0.058
buy: 20180912-03-01 15926 2 31.4 0.056
buy: 20180721-16-11 15931 5 26.2 0.067
buy: 20180315-19-02 15932 2 19.3 0.094
buy: 20180123-19-09 15933 0 20.4 0.086
buy: 20180123-19-09 15933 9 32.1 0.064
buy: 20180909-06-06 15934 2 36.3 0.062
buy: 20181127-10-05 15937 70 49.0 0.057
buy: 20180705-21-08 15938 0 17.2 0.09
buy: 20181130-23-06 15941 5 26.7 0.06
buy: 20181130-23-06 15

buy: 20171028-19-05 16430 2 31.9 0.055
buy: 20180116-17-10 16437 3 25.7 0.061
buy: 20181021-12-01 16440 1 30.8 0.065
buy: 20171226-22-06 16441 9 30.7 0.054
buy: 20181004-12-01 16449 1 27.4 0.057
buy: 20180225-16-12 16450 8 22.1 0.08
buy: 20181205-18-10 16459 21 24.1 0.092
buy: 20180323-03-10 16465 12 26.7 0.058
buy: 20180526-06-10 16467 4 18.9 0.079
buy: 20180526-06-10 16467 17 30.3 0.055
buy: 20180518-21-07 16481 25 64.1 0.061
buy: 20180129-24-11 16483 8 33.3 0.059
buy: 20180514-03-09 16484 3 29.7 0.078
buy: 20180312-04-09 16485 20 32.5 0.05
buy: 20180801-04-02 16486 4 17.3 0.095
buy: 20181102-24-10 16489 2 23.1 0.068
buy: 20181215-10-03 16490 22 26.5 0.068
buy: 20180531-05-05 16491 0 31.9 0.055
☆hit!☆: 20180531-05-05 0 31.9
buy: 20180531-05-05 16491 2 42.7 0.063
buy: 20180531-05-05 16491 12 36.7 0.053
buy: 20180502-10-10 16495 10 29.7 0.065
buy: 20180502-24-03 16497 0 15.8 0.098
buy: 20171208-03-05 16503 1 34.3 0.069
buy: 20171208-03-05 16503 2 32.2 0.072
buy: 20171208-03-05 16503 8 

buy: 20171006-13-09 16977 4 29.6 0.059
buy: 20180417-19-01 16982 1 43.7 0.052
buy: 20180417-19-01 16982 8 57.7 0.051
buy: 20180626-24-02 16985 0 14.5 0.142
☆hit!☆: 20180626-24-02 0 14.5
buy: 20180320-08-08 16987 1 25.4 0.076
buy: 20180906-22-12 16991 0 16.7 0.091
buy: 20180515-09-04 16997 43 52.0 0.059
buy: 20180704-24-02 16999 1 37.3 0.062
buy: 20180704-24-02 16999 4 59.6 0.055
buy: 20180704-24-02 16999 5 27.0 0.072
buy: 20180526-10-02 17001 2 34.0 0.062
buy: 20181104-02-03 17003 0 39.4 0.078
buy: 20181104-02-03 17003 1 28.1 0.06
buy: 20180423-09-07 17005 1 22.0 0.083
buy: 20180423-09-07 17005 8 29.7 0.067
buy: 20180513-17-08 17009 5 27.3 0.056
buy: 20180503-05-10 17010 5 22.6 0.072
buy: 20180609-14-11 17014 2 55.8 0.055
buy: 20180419-14-01 17016 1 28.4 0.061
buy: 20180419-14-01 17016 2 68.8 0.053
buy: 20180918-01-09 17017 9 30.9 0.054
buy: 20171224-01-06 17018 0 25.9 0.063
buy: 20180617-14-03 17025 2 18.9 0.098
buy: 20180821-22-11 17026 0 31.0 0.052
buy: 20171109-03-01 17031 4 25.4 0

buy: 20170924-10-05 17511 0 22.5 0.074
buy: 20180228-16-06 17520 1 27.3 0.06
buy: 20171216-02-06 17521 0 25.2 0.07
buy: 20171217-22-07 17522 7 26.8 0.075
buy: 20171217-22-07 17522 12 52.6 0.059
buy: 20171227-03-11 17524 0 19.8 0.086
buy: 20171227-03-11 17524 3 20.5 0.081
buy: 20180225-04-09 17526 0 16.0 0.1
buy: 20180225-04-09 17526 4 37.4 0.066
buy: 20180225-04-09 17526 5 46.4 0.062
buy: 20171025-10-05 17528 1 87.6 0.058
buy: 20180113-06-06 17529 20 11.0 0.162
buy: 20171216-11-10 17530 7 24.6 0.062
buy: 20181031-22-11 17531 3 33.5 0.051
buy: 20180618-02-11 17532 12 22.8 0.067
buy: 20180715-05-09 17534 0 35.1 0.085
buy: 20180715-05-09 17534 4 25.9 0.069
buy: 20180805-20-09 17539 18 46.9 0.059
buy: 20180911-19-03 17540 36 42.8 0.055
buy: 20180531-06-01 17543 0 24.2 0.068
buy: 20180531-06-01 17543 1 24.7 0.062
buy: 20180531-06-01 17543 4 21.4 0.084
buy: 20171003-24-02 17550 5 30.7 0.053
buy: 20180219-15-07 17559 3 35.4 0.059
buy: 20180219-15-07 17559 23 38.6 0.07
buy: 20180213-17-03 1756

In [21]:
print("resultReturn:",resAmount/buyAmount)
print("totalRace,buy,return",len(y_test),buyAmount,resAmount )

resultReturn: 0.9063456408816514
totalRace,buy,return 17925 7123 6455.900000000002


In [22]:
# trainの回収率を計算
y_pred = model.predict(X_train, num_iteration=model.best_iteration)
y_pred_max = np.argmax(y_pred, axis=1)  # 最尤と判断したクラスの値にする

#print(X_train.head())
#print(y_train[0:5])
#print(y_pred_max[0:5])
#c = collections.Counter(y_pred_max)
#print(len(c) )

# 精度 (Accuracy) を計算する
# エラーになるけど気にしない。
#accuracy = sum(y_train == y_pred_max) / len(y_train)
#print("accuracy:",accuracy)

# 回収率を計算
#res=0
#for i in range(len(y_train)):
#    if y_train.iloc[i]==y_pred_max[i]:
#        res += o_train[i] -1
#    else:
#        pass
#print("return:",res/len(y_train))


In [23]:
print(y_pred[0])

[0.0953807  0.10174937 0.05685786 0.04171899 0.04999769 0.03833318
 0.03476708 0.01441034 0.05026547 0.01768751 0.02297994 0.0139283
 0.03098234 0.01812042 0.01614222 0.01380845 0.00843881 0.00817225
 0.01112992 0.01070895 0.03520299 0.028697   0.03563778 0.01292432
 0.00712439 0.00561997 0.00468562 0.00363059 0.01146743 0.0033043
 0.00344275 0.00409637 0.00547834 0.00209078 0.00191154 0.00518028
 0.00229842 0.00328768 0.00220057 0.00080013 0.00490551 0.01066108
 0.00590217 0.00212045 0.00545642 0.00662634 0.00273996 0.00146455
 0.00285901 0.00204076 0.00142158 0.00076187 0.00223162 0.00095865
 0.00263442 0.00153449 0.00169911 0.00124804 0.00038724 0.00075035
 0.00858545 0.00496459 0.00368331 0.00311538 0.00502546 0.00230074
 0.00282259 0.00150994 0.0017871  0.00082219 0.00157036 0.00042565
 0.00645572 0.00171765 0.00117089 0.00148711 0.00141446 0.00096428
 0.00071167 0.00111202 0.00420797 0.00237499 0.00482866 0.00370502
 0.00511119 0.00086561 0.00086773 0.00115857 0.00185883 0.002131

In [24]:
print(df.iloc[X_train.index]['raceId'])

60038    20181015-05-08
15915    20171223-05-11
43991    20180624-20-09
368      20170903-23-10
141      20170902-07-11
40587    20180602-11-07
14112    20171210-14-10
16309    20171225-10-12
37924    20180516-10-04
49820    20180803-20-12
38878    20180522-18-09
10549    20171116-20-11
42214    20180612-12-11
30516    20180327-13-01
89       20170901-23-07
15726    20171221-22-02
70429    20181224-20-08
19102    20180108-13-12
67027    20181203-06-04
14016    20171210-03-10
62779    20181104-11-05
43266    20180619-20-01
53468    20180827-13-12
25660    20180224-14-12
15085    20171217-04-07
6963     20171022-14-07
35523    20180430-24-09
45222    20180703-07-02
15677    20171221-12-01
44293    20180626-23-02
              ...      
66636    20181130-13-05
37625    20180514-02-09
59585    20181012-02-05
273      20170903-05-11
10263    20171114-19-12
31749    20180406-04-12
47151    20180716-01-11
47541    20180718-24-10
50024    20180805-08-06
22450    20180131-12-01
4193     2017100

In [25]:
print(xdf.columns)

Index(['l1Fcnt', 'l1boat2r', 'l1boat3r', 'l1motor2r', 'l1motor3r',
       'l1oldavgstdev', 'l1oldavgsttime', 'l1oldrank1', 'l1oldrank2',
       'l1oldrank3', 'l1oldrank4', 'l1oldrank5', 'l1oldrank6', 'l1rank',
       'l2Fcnt', 'l2boat2r', 'l2boat3r', 'l2motor2r', 'l2motor3r',
       'l2oldavgstdev', 'l2oldavgsttime', 'l2oldrank1', 'l2oldrank2',
       'l2oldrank3', 'l2oldrank4', 'l2oldrank5', 'l2oldrank6', 'l2rank',
       'l3Fcnt', 'l3boat2r', 'l3boat3r', 'l3motor2r', 'l3motor3r',
       'l3oldavgstdev', 'l3oldavgsttime', 'l3oldrank1', 'l3oldrank2',
       'l3oldrank3', 'l3oldrank4', 'l3oldrank5', 'l3oldrank6', 'l3rank',
       'l4Fcnt', 'l4boat2r', 'l4boat3r', 'l4motor2r', 'l4motor3r',
       'l4oldavgstdev', 'l4oldavgsttime', 'l4oldrank1', 'l4oldrank2',
       'l4oldrank3', 'l4oldrank4', 'l4oldrank5', 'l4oldrank6', 'l4rank',
       'l5Fcnt', 'l5boat2r', 'l5boat3r', 'l5motor2r', 'l5motor3r',
       'l5oldavgstdev', 'l5oldavgsttime', 'l5oldrank1', 'l5oldrank2',
       'l5oldrank3', 'l

In [26]:
print(model.feature_importance())

[  42  311  392  482  616  444  763 2290 1029  457  472  835 1081  538
   17  399  316  592  628  442  602 1873  862  445  294  894 1167  527
   24  453  393  671  738  512  698 2045  839  492  359  795 1049  501
   10  345  348  633  588  455  747 2450 1072  635  354  646 1230  427
   14  408  421  624  661  471  840 2449 1489  765  405  423 1054  630
   35  325  336  552  602  539  700 2753 1413  815  381  469 1125  596
  157  229]
